In [14]:
#import important libraries.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas.api.types import is_numeric_dtype
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error, accuracy_score
import math
import warnings
warnings.filterwarnings('ignore')

In [15]:
#read dataset csv file.
df = pd.read_csv("bank.csv", delimiter=";", header='infer')

In [16]:
# find the features and observation in the data set.
df.shape

(4521, 17)

In [17]:
# find the datatypes of the features and target variable. 
df.dtypes

age           int64
job          object
marital      object
education    object
default      object
balance       int64
housing      object
loan         object
contact      object
day           int64
month        object
duration      int64
campaign      int64
pdays         int64
previous      int64
poutcome     object
y            object
dtype: object

In [18]:
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,30,unemployed,married,primary,no,1787,no,no,cellular,19,oct,79,1,-1,0,unknown,no
1,33,services,married,secondary,no,4789,yes,yes,cellular,11,may,220,1,339,4,failure,no
2,35,management,single,tertiary,no,1350,yes,no,cellular,16,apr,185,1,330,1,failure,no
3,30,management,married,tertiary,no,1476,yes,yes,unknown,3,jun,199,4,-1,0,unknown,no
4,59,blue-collar,married,secondary,no,0,yes,no,unknown,5,may,226,1,-1,0,unknown,no


In [19]:
bank_data_new = pd.get_dummies(df, columns=['job', 'marital', 'education', 'default', 'housing', 'loan',
                                                   'contact', 'month', 'poutcome'])

bank_data_new.head()

,age,balance,day,duration,campaign,pdays,previous,y,job_admin.,job_blue-collar,...,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,poutcome_failure,poutcome_other,poutcome_success,poutcome_unknown
0,30,1787,19,79,1,-1,0,no,0,0,...,0,0,0,0,1,0,0,0,0,1
1,33,4789,11,220,1,339,4,no,0,0,...,0,0,1,0,0,0,1,0,0,0
2,35,1350,16,185,1,330,1,no,0,0,...,0,0,0,0,0,0,1,0,0,0
3,30,1476,3,199,4,-1,0,no,0,0,...,1,0,0,0,0,0,0,0,0,1
4,59,0,5,226,1,-1,0,no,0,1,...,0,0,1,0,0,0,0,0,0,1


In [20]:
bank_data_new.y.replace(('yes', 'no'), (1, 0), inplace=True)


In [21]:
y = pd.DataFrame(bank_data_new['y'])

In [22]:
X = bank_data_new.drop(['y'], axis=1)

In [23]:
X.shape, y.shape

((4521, 51), (4521, 1))

In [24]:
#Using Python Code 
class LinearTrainer:

    def __init__(self):

        # Learning Rate
        self.l_rate = 0.001

        # Total iterations
        self.iterations = 60

    def trains(self, x_data_train, y_data_train):
        number_of_layers_list = [x_data_train.shape[1], 30, 50, 40, 20, 1]
        iteration_value = 0
        weights_value = []
        biased_value = []
        z = [0] * 6
        y_data_train_predicted = [0] * 6
        cost_function = [0] * 6
        da = [0] * 6
        dz = [0] * 6
        dw = [0] * 6
        db = [0] * 6
        y_data_train_predicted[0] = x_data_train.T
        data_set_size = x_data_train.shape[0]

        for k in range(1, 6):
            weights_value.append(np.random.rand(number_of_layers_list[k], number_of_layers_list[k-1]) * 0.01)
            biased_value.append(np.zeros((number_of_layers_list[k], 1), dtype='float'))

        while iteration_value < self.iterations:
            j = 0

            while j < 5:
                # z = w.T*x + b and z matrix shape = (1 x data_set_size)
                z[j] = np.dot(weights_value[j], y_data_train_predicted[j]) + biased_value[j]

                # y^_train = sigmoid(z) and y^ matrix shape = (1 x data_set_size)
                y_data_train_predicted[j+1] = 1 / 1 + np.exp(-z[j])

                cost_function[j] = (1 / (1 + np.exp(-z[j]))) * (1 - (1 / (1 + np.exp(-z[j]))))

                # dA[l-1] = w[l].T * dZ[l]
                da[j] = (-(y_data_train.T / y_data_train_predicted[j+1]) + ((1 - y_data_train.T) / (1 - y_data_train_predicted[j+1])))

                # dz = y^_train - y and dz matrix shape = (1 x data_set_size)
                dz[j] = da[j] * cost_function[j]

                # dw = (x * dz.T ) / data_set_size and dw matrix shape = (features_size x 1)
                dw[j] = np.dot(dz[j], (np.transpose(y_data_train_predicted[j])))/data_set_size

                # db = (dz.sum ) / data_set_size and db matrix shape = (1 x 1)
                db[j] = np.sum(dz[j], axis=1, keepdims=True) / data_set_size

                # w = w - alpha * dw and w matrix shape = (features_size x 1)
                weights_value[j] = weights_value[j] - np.dot(self.l_rate, dw[j])

                # b = b - alpha * db and b matrix shape = (features_size x 1)
                biased_value[j] = biased_value[j] - np.dot(self.l_rate, db[j])

                j += 1

            iteration_value += 1

        return weights_value, biased_value

    def classify(self, x_data_test, parameters):
        j = 0
        z = [0] * 6
        y_data_test_predicted = [0] * 6
        y_data_test_predicted[0] = x_data_test.T

        while j < 5:

            # z = w.T*x + b and z matrix shape = (1 x data_set_size)
            z[j] = np.dot(parameters[0][j], y_data_test_predicted[j]) + parameters[1][j]

            # y^_test = sigmoid(z) and y^ matrix shape = (1 x data_set_size)
            y_data_test_predicted[j+1] = 1 / 1 + np.exp(-z[j])

            j += 1

        return y_data_test_predicted[-1]

    def accuracy(self, y_data_test, y_pred_test):
        y_pred_test = np.nan_to_num(y_pred_test)
        y_pred_test = y_pred_test.T
        accuracy = mean_absolute_error(y_data_test, y_pred_test)
        return (1 - accuracy) * 100


def main():
    l_t = LinearTrainer()
    x_data = np.array(X)
    y_data = np.array(y)
    y_data = y_data.reshape(len(y_data), 1)

    x_data_train, x_data_test, y_data_train, y_data_test = train_test_split(x_data, y_data
                                                                            , test_size=0.3, random_state=42)

    l_t = LinearTrainer()
    parameters = l_t.trains(x_data_train, y_data_train)
    y_prediction = l_t.classify(x_data_test, parameters)
    y_prediction_train = l_t.classify(x_data_train, parameters)
    accuracy = l_t.accuracy(y_data_test, y_prediction)
    accuracy_train = l_t.accuracy(y_data_train, y_prediction_train)

    print("Train accuracy: ", accuracy_train)
    print("Test accuracy: ", accuracy)



if __name__ == '__main__':
    main()


Train accuracy:  88.33754740834387
Test accuracy:  88.7988209285188


In [25]:
# Using sklearn linear regression model
x_data_train, x_data_test, y_data_train, y_data_test = train_test_split(X, y, test_size=0.3, random_state=0)
model = MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(5, 2), random_state=1)
reg = model.fit(x_data_train,y_data_train)
reg.score(x_data_test, y_data_test)
y_pred =  reg.predict(x_data_test)
print("Train accuracy", accuracy_score(y_data_train, y_data_train) * 100)
print("Test accuracy", accuracy_score(y_data_test, y_pred) * 100)

Train accuracy 100.0
Test accuracy 87.61974944731024
